In [1623]:
import warnings
import numpy as np
import pandas as pd
import scipy.stats as st
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.plotly as py
import plotly.graph_objs as go

from scipy.stats import skew
from math import sqrt
from plotly import tools
from plotly.offline import init_notebook_mode, iplot


from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import PowerTransformer, OneHotEncoder
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.linear_model import Lasso, LassoCV, Ridge, 
from sklearn.linear_model import RidgeCV, ElasticNet, ElasticNetCV
from sklearn.linear_model import LassoLarsIC, BayesianRidge
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.base import TransformerMixin, clone
from sklearn.metrics import mean_squared_error

import xgboost as xgb
import lightgbm as lgb

from hyperopt import hp
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

init_notebook_mode(connected=True)
%matplotlib inline
sns.set()
warnings.filterwarnings('ignore')
pd.set_option('display.float_format', 
              lambda x: '{:.5f}'.format(x)) #Limiting floats output to 5 decimal points

In [1624]:
def rmsle_cv(model, X, y):
    ''' cross_val_score with shuffled data before splitting. 
        Return Root-Mean-Squared-Error (RMSE) '''

    kf = KFold(n_splits=5, shuffle=True,
               random_state=42).get_n_splits(X.values)

    rmse = np.sqrt(-cross_val_score(model,
                                    X.values,
                                    y,
                                    scoring='neg_mean_squared_error',
                                    cv=kf))
    return(rmse)


def write_to_submission_file(predicted_labels, out_file,
                             target='SalePrice', index_label="Id"):
    predicted_df = pd.DataFrame(predicted_labels,
                                index=np.arange(
                                    1461, predicted_labels.shape[0] + 1461),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [1674]:
X_train = pd.read_csv('data/X_train.csv', index_col='Id')
X_test = pd.read_csv('data/X_test.csv', index_col='Id')

y_train = X_train[['SalePrice']]
X_train.drop(['SalePrice'], axis=1, inplace=True)

In [1681]:
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,258,259,260,261,262,263,264,265,266,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,0.49522,-0.16276,-0.17970,0.67449,-0.47866,1.15219,0.91007,1.22321,0.62422,-0.35683,...,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,12.24770
2,-1.16293,0.55604,0.08625,-0.03845,2.00874,0.00691,-0.61869,-0.82556,1.25410,-0.35683,...,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,12.10902
3,0.49522,-0.01343,0.42854,0.67449,-0.47866,1.05566,0.83773,1.18539,0.11475,-0.35683,...,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,12.31717
4,0.69982,-0.41878,0.07520,0.67449,-0.47866,-1.62874,-0.86076,-0.82556,-0.51051,-0.35683,...,0.00000,0.00000,1.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,11.84940
5,0.49522,0.73710,0.96546,1.36238,-0.47866,1.00815,0.69726,1.32862,0.50611,-0.35683,...,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,12.42922


## Finding base parametrs for models

### Lasso Regression:

Trying different spaces of alphas

In [1237]:
n_alphas = 1000
alphas = np.linspace(0.00000001, 0.3, n_alphas)

kf = KFold(n_splits=5,
           shuffle=True,
           random_state=42).get_n_splits(X_train.values)

lasso_cv = LassoCV(alphas=alphas,
                   cv=kf,
                   random_state=42,
                   n_jobs=-1)

lasso_cv.fit(X_train, y_train)

lasso_cv_rmse = np.sqrt((np.mean(-cross_val_score(Lasso(lasso_cv.alpha_), 
                                                  X_train, 
                                                  y_train, 
                                                  cv=kf,
                                                  scoring='neg_mean_squared_error'))))

lasso_best_alpha = lasso_cv.alpha_

print('Lasso best score: ' + str(lasso_cv_rmse))
print('Lasso best alpha: ' + str(lasso_best_alpha))

Lasso best score: 0.108996510864527
Lasso best alpha: 0.0006006105805805806


First trial:  
Lasso best score: 0.2833659534110847  
Lasso best alpha: 0.0015075475376884423

Second trial:  
Lasso best score: 0.108996510864527  
Lasso best alpha: 0.0006006105805805806

### Ridge Regression:

Trying different spaces of alphas

In [1238]:
n_alphas = 1000
ridge_alphas = np.linspace(-1, 40, n_alphas)

ridge_cv = RidgeCV(alphas=ridge_alphas,
                   scoring='neg_mean_squared_error',
                   cv=kf)

ridge_cv.fit(X_train, y_train)

ridge_cv_rmse = np.sqrt((np.mean(-cross_val_score(Ridge(ridge_cv.alpha_),
                                                  X_train,
                                                  y_train,
                                                  cv=kf,
                                                  scoring='neg_mean_squared_error'))))

ridge_best_alpha = ridge_cv.alpha_

print('Ridge best score: ' + str(ridge_cv_rmse))
print('Ridge best alpha: ' + str(ridge_best_alpha))

Ridge best score: 0.11186647061455098
Ridge best alpha: 18.986986986986985


First trial:  
Ridge best score: 0.2916281960056402  
Ridge best alpha: 20.858585858585858

Second:  
Ridge best score: 0.11186647061455098  
Ridge best alpha: 18.986986986986985

### Elastic Net Regression:

In [1239]:
n_alphas = 1000
elastic_l1_ratio = np.linspace(0.1, 0.9, 100)
elastic_alphas = np.linspace(0.001, 0.2, n_alphas)


elastic_cv = ElasticNetCV(l1_ratio=elastic_l1_ratio,
                          alphas=elastic_alphas,
                          cv=kf,
                          n_jobs=-1)

elastic_cv.fit(X_train, y_train)

elastic_cv_rmse = np.sqrt((np.mean(-cross_val_score(ElasticNet(elastic_cv.alpha_), 
                                                    X_train, 
                                                    y_train, 
                                                    cv=kf,
                                                    scoring='neg_mean_squared_error'))))

elastic_best_alpha = elastic_cv.alpha_
elastic_best_l1 = elastic_cv.l1_ratio_

print('Elastic best score: ' + str(elastic_cv_rmse))
print('Elastic best L1: ' + str(elastic_best_l1))
print('Elastic best alpha: ' + str(elastic_best_alpha))

Elastic best score: 0.10922445993923736
Elastic best L1: 0.6414141414141414
Elastic best alpha: 0.001


First trial:  
Elastic best score: 0.284775909931625  
Elastic best L1: 0.5222222222222221  
Elastic best alpha: 0.00301010101010101

Second:  
Elastic best score: 0.10922445993923736  
Elastic best L1: 0.6414141414141414  
Elastic best alpha: 0.001

### Kernel Ridge Regression:

In [1240]:
n_alphas = 100
krr_params = {'alpha': np.linspace(0.0001, 10, n_alphas),
              'kernel': ['linear', 'polynomial'],
              'degree': np.arange(2, 5),
              'coef0': np.arange(0, 10, 1)}

krr_gcv = GridSearchCV(KernelRidge(),
                       krr_params,
                       cv=kf,
                       scoring='neg_mean_squared_error',
                       n_jobs=-1,
                       verbose=True)


krr_gcv.fit(X_train, y_train)

Fitting 5 folds for each of 6000 candidates, totalling 30000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   12.6s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   42.5s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 1792 tasks      | elapsed:  5.9min
[Parallel(n_jobs=-1)]: Done 2442 tasks      | elapsed:  8.0min
[Parallel(n_jobs=-1)]: Done 3192 tasks      | elapsed: 10.5min
[Parallel(n_jobs=-1)]: Done 4042 tasks      | elapsed: 13.3min
[Parallel(n_jobs=-1)]: Done 4992 tasks      | elapsed: 16.4min
[Parallel(n_jobs=-1)]: Done 6042 tasks      | elapsed: 19.8min
[Parallel(n_jobs=-1)]: Done 7192 tasks      | elapsed: 23.6min
[Parallel(n_jobs=-1)]: Done 8442 tasks      | elapsed: 27.7min
[Parallel(n_jobs=-1)]: Done 9792 tasks      | elapsed: 32.2min
[Parallel(n_jobs=-1)]: Done 11242 tasks      |

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=KernelRidge(alpha=1, coef0=1, degree=3, gamma=None, kernel='linear',
      kernel_params=None),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'alpha': array([1.00000e-04, 1.01109e-01, ..., 9.89899e+00, 1.00000e+01]), 'kernel': ['linear', 'polynomial'], 'degree': array([2, 3, 4]), 'coef0': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=True)

In [1241]:
krr_cv_rmse = np.sqrt((np.mean(-cross_val_score(KernelRidge(alpha=2.2223, 
                                                            coef0=2, 
                                                            degree=4, 
                                                            kernel='polynomial'), 
                                                X_train, 
                                                y_train, 
                                                cv=kf,
                                                scoring='neg_mean_squared_error'))))

krr_best_alpha = krr_gcv.best_params_['alpha']
krr_best_coef0 = krr_gcv.best_params_['coef0']
krr_best_degree = krr_gcv.best_params_['degree']
krr_best_kernel = krr_gcv.best_params_['kernel']

print('Kernel Ridge Regression best score: ' + str(krr_cv_rmse))
print('Kernel Ridge Regression best alpha: ' + str(krr_best_alpha))
print('Kernel Ridge Regression best coef0: ' + str(krr_best_coef0))
print('Kernel Ridge Regression best degree: ' + str(krr_best_degree))
print('Kernel Ridge Regression best kernel: ' + str(krr_best_kernel))

Kernel Ridge Regression best score: 0.11096877626979507
Kernel Ridge Regression best alpha: 1.4142272727272727
Kernel Ridge Regression best coef0: 3
Kernel Ridge Regression best degree: 3
Kernel Ridge Regression best kernel: polynomial


In [1242]:
krr_gcv.best_params_

{'alpha': 1.4142272727272727, 'coef0': 3, 'degree': 3, 'kernel': 'polynomial'}

First trial:  
score: 0.2851655918827155  
{'alpha': 2.2223, 'coef0': 2, 'degree': 4, 'kernel': 'polynomial'}

Second:  
score: 0.11096877626979507  
{'alpha': 1.4142272727272727, 'coef0': 3, 'degree': 3, 'kernel': 'polynomial'}

### Gradient Boosting Regression:

With huber loss that makes it robust to outliers

In [1249]:
X_train_gb, X_valid_gb, y_train_gb, y_valid_gb = train_test_split(X_train,
                                                                  y_train,
                                                                  test_size=0.3,
                                                                  random_state=42)

In [1259]:
def score_GB(params):
    print("Training with params:")
    print(params)
    params['max_depth'] = int(params['max_depth'])
    params['min_samples_split'] = int(params['min_samples_split'])
    params['min_samples_leaf'] = int(params['min_samples_leaf'])
    model = GradientBoostingRegressor(**params)
    model.fit(X_train_gb, y_train_gb)
    predictions = model.predict(X_valid_gb)
    score = np.sqrt(mean_squared_error(y_valid_gb, predictions))
    print("\tScore {0}\n\n".format(score))
    loss = score
    return {'loss': loss, 'status': STATUS_OK}


def optimize_GB(trials):
    space = {
        'loss': 'huber',
        'learning_rate': hp.quniform('learning_rate', 0.005, 0.05, 0.005),
        'n_estimators': 100,
        'subsample': hp.quniform('subsample', 0.5, 1, 0.05),
        'min_samples_split': hp.quniform('min_samples_split', 2, 30, 1),
        'min_samples_leaf': hp.quniform('min_samples_leaf', 2, 20, 1),
        'max_depth': hp.quniform('max_depth', 2, 20, 1),
        'max_features': hp.quniform('max_features', 0.3, 1, 0.05),
        'alpha': hp.quniform('alpha', 0.1, 0.9, 0.05),
    }

    best = fmin(score_GB, space, algo=tpe.suggest,
                trials=trials, max_evals=1000)
    return best

In [1260]:
trials = Trials()
best_params_GB = optimize_GB(trials)
best_params_GB

Training with params:                                 
{'alpha': 0.35000000000000003, 'learning_rate': 0.035, 'loss': 'huber', 'max_depth': 17.0, 'max_features': 0.9, 'min_samples_leaf': 7.0, 'min_samples_split': 30.0, 'n_estimators': 100, 'subsample': 0.8500000000000001}
	Score 0.14826380428393648                            


Training with params:                                                             
{'alpha': 0.1, 'learning_rate': 0.045, 'loss': 'huber', 'max_depth': 6.0, 'max_features': 0.75, 'min_samples_leaf': 11.0, 'min_samples_split': 3.0, 'n_estimators': 100, 'subsample': 0.75}
	Score 0.14357471532666002                                                        


Training with params:                                                             
{'alpha': 0.4, 'learning_rate': 0.025, 'loss': 'huber', 'max_depth': 9.0, 'max_features': 0.9500000000000001, 'min_samples_leaf': 19.0, 'min_samples_split': 9.0, 'n_estimators': 100, 'subsample': 0.75}
	Score 0.16346459291447896   

	Score 0.15179527392792233                                                         


Training with params:                                                              
{'alpha': 0.35000000000000003, 'learning_rate': 0.04, 'loss': 'huber', 'max_depth': 9.0, 'max_features': 0.7000000000000001, 'min_samples_leaf': 9.0, 'min_samples_split': 22.0, 'n_estimators': 100, 'subsample': 0.7000000000000001}
	Score 0.14425036101422906                                                         


Training with params:                                                              
{'alpha': 0.8, 'learning_rate': 0.03, 'loss': 'huber', 'max_depth': 7.0, 'max_features': 0.9500000000000001, 'min_samples_leaf': 7.0, 'min_samples_split': 26.0, 'n_estimators': 100, 'subsample': 0.9500000000000001}
	Score 0.1506572465960763                                                          


Training with params:                                                              
{'alpha': 0.65, 'learning_rate': 0.045, 'los

	Score 0.14034417383316533                                                       


Training with params:                                                            
{'alpha': 0.65, 'learning_rate': 0.05, 'loss': 'huber', 'max_depth': 7.0, 'max_features': 0.4, 'min_samples_leaf': 12.0, 'min_samples_split': 22.0, 'n_estimators': 100, 'subsample': 0.6000000000000001}
	Score 0.1382681260006351                                                        


Training with params:                                                            
{'alpha': 0.8500000000000001, 'learning_rate': 0.045, 'loss': 'huber', 'max_depth': 3.0, 'max_features': 0.35000000000000003, 'min_samples_leaf': 10.0, 'min_samples_split': 17.0, 'n_estimators': 100, 'subsample': 0.5}
	Score 0.1406062072534528                                                        


Training with params:                                                            
{'alpha': 0.6000000000000001, 'learning_rate': 0.05, 'loss': 'huber', 'max_depth':

	Score 0.15298228744841674                                                          


Training with params:                                                               
{'alpha': 0.30000000000000004, 'learning_rate': 0.045, 'loss': 'huber', 'max_depth': 14.0, 'max_features': 0.5, 'min_samples_leaf': 13.0, 'min_samples_split': 16.0, 'n_estimators': 100, 'subsample': 0.65}
	Score 0.14160427296982311                                                          


Training with params:                                                               
{'alpha': 0.7000000000000001, 'learning_rate': 0.04, 'loss': 'huber', 'max_depth': 20.0, 'max_features': 0.45, 'min_samples_leaf': 14.0, 'min_samples_split': 23.0, 'n_estimators': 100, 'subsample': 0.75}
	Score 0.140740706061287                                                            


Training with params:                                                               
{'alpha': 0.45, 'learning_rate': 0.05, 'loss': 'huber', 'max_depth': 19.0, 

	Score 0.13790056563350903                                                          


Training with params:                                                               
{'alpha': 0.5, 'learning_rate': 0.04, 'loss': 'huber', 'max_depth': 17.0, 'max_features': 0.55, 'min_samples_leaf': 4.0, 'min_samples_split': 25.0, 'n_estimators': 100, 'subsample': 0.5}
	Score 0.1424273289801634                                                           


Training with params:                                                               
{'alpha': 0.45, 'learning_rate': 0.045, 'loss': 'huber', 'max_depth': 12.0, 'max_features': 0.45, 'min_samples_leaf': 11.0, 'min_samples_split': 22.0, 'n_estimators': 100, 'subsample': 0.55}
	Score 0.14113161023275264                                                          


Training with params:                                                               
{'alpha': 0.65, 'learning_rate': 0.035, 'loss': 'huber', 'max_depth': 15.0, 'max_features': 0.600000000000

...

	Score 0.14005355741268938                                                         


Training with params:                                                              
{'alpha': 0.75, 'learning_rate': 0.045, 'loss': 'huber', 'max_depth': 20.0, 'max_features': 0.30000000000000004, 'min_samples_leaf': 11.0, 'min_samples_split': 7.0, 'n_estimators': 100, 'subsample': 0.6000000000000001}
	Score 0.13429919730856157                                                         


Training with params:                                                              
{'alpha': 0.9, 'learning_rate': 0.05, 'loss': 'huber', 'max_depth': 18.0, 'max_features': 0.45, 'min_samples_leaf': 10.0, 'min_samples_split': 16.0, 'n_estimators': 100, 'subsample': 0.7000000000000001}
	Score 0.13503208536822986                                                         


Training with params:                                                              
{'alpha': 0.8500000000000001, 'learning_rate': 0.05, 'loss': 'huber'

	Score 0.13674063260849392                                                         


Training with params:                                                              
{'alpha': 0.65, 'learning_rate': 0.045, 'loss': 'huber', 'max_depth': 17.0, 'max_features': 0.4, 'min_samples_leaf': 6.0, 'min_samples_split': 8.0, 'n_estimators': 100, 'subsample': 0.6000000000000001}
	Score 0.1355904929721442                                                          


Training with params:                                                              
{'alpha': 0.8500000000000001, 'learning_rate': 0.05, 'loss': 'huber', 'max_depth': 18.0, 'max_features': 0.45, 'min_samples_leaf': 9.0, 'min_samples_split': 29.0, 'n_estimators': 100, 'subsample': 0.7000000000000001}
	Score 0.13661617386290323                                                         


Training with params:                                                              
{'alpha': 0.8500000000000001, 'learning_rate': 0.04, 'loss': 'huber', '

	Score 0.13666380275451134                                                         


Training with params:                                                              
{'alpha': 0.8500000000000001, 'learning_rate': 0.05, 'loss': 'huber', 'max_depth': 17.0, 'max_features': 0.45, 'min_samples_leaf': 11.0, 'min_samples_split': 3.0, 'n_estimators': 100, 'subsample': 0.65}
	Score 0.13381253265797782                                                         


Training with params:                                                              
{'alpha': 0.9, 'learning_rate': 0.045, 'loss': 'huber', 'max_depth': 19.0, 'max_features': 0.4, 'min_samples_leaf': 10.0, 'min_samples_split': 4.0, 'n_estimators': 100, 'subsample': 0.6000000000000001}
	Score 0.13424504881756036                                                         


Training with params:                                                              
{'alpha': 0.9, 'learning_rate': 0.05, 'loss': 'huber', 'max_depth': 15.0, 'max_featur

	Score 0.13436484381282618                                                         


Training with params:                                                              
{'alpha': 0.9, 'learning_rate': 0.045, 'loss': 'huber', 'max_depth': 16.0, 'max_features': 0.4, 'min_samples_leaf': 10.0, 'min_samples_split': 2.0, 'n_estimators': 100, 'subsample': 0.65}
	Score 0.13550313675116044                                                         


Training with params:                                                              
{'alpha': 0.8500000000000001, 'learning_rate': 0.05, 'loss': 'huber', 'max_depth': 14.0, 'max_features': 0.30000000000000004, 'min_samples_leaf': 4.0, 'min_samples_split': 27.0, 'n_estimators': 100, 'subsample': 0.65}
	Score 0.13422120249497346                                                         


Training with params:                                                              
{'alpha': 0.8, 'learning_rate': 0.05, 'loss': 'huber', 'max_depth': 19.0, 'max_featu

{'alpha': 0.9,
 'learning_rate': 0.045,
 'max_depth': 18.0,
 'max_features': 0.4,
 'min_samples_leaf': 8.0,
 'min_samples_split': 4.0,
 'subsample': 0.6000000000000001}

In [1261]:
best_params_GB['max_depth'] = int(best_params_GB['max_depth'])
best_params_GB['min_samples_leaf'] = int(best_params_GB['min_samples_leaf'])
best_params_GB['min_samples_split'] = int(best_params_GB['min_samples_split'])

[el + '=' + str(best_params_GB[el]) for el in best_params_GB]

['alpha=0.9',
 'learning_rate=0.045',
 'max_depth=18',
 'max_features=0.4',
 'min_samples_leaf=8',
 'min_samples_split=4',
 'subsample=0.6000000000000001']

First trial:  
score: 0.3034719322867967  
['alpha=0.9',
 'learning_rate=0.05',
 'max_depth=5',
 'max_features=0.75',
 'min_samples_leaf=5',
 'min_samples_split=17',
 'subsample=0.65']

Second:  
Score 0.1310445071349567  
{'alpha': 0.9, 'learning_rate': 0.045, 'loss': 'huber', 'max_depth': 18.0, 'max_features': 0.4, 'min_samples_leaf': 8.0, 'min_samples_split': 4.0, 'n_estimators': 100, 'subsample': 0.6000000000000001}

In [378]:
gb_rmse = rmsle_cv(GradientBoostingRegressor(n_estimators=3000,
                                             loss='huber',
                                             alpha=0.9,
                                             learning_rate=0.05,
                                             max_depth=5,
                                             max_features=0.75,
                                             min_samples_leaf=5,
                                             min_samples_split=17,
                                             subsample=0.65,
                                             random_state=42,
                                             n_iter_no_change=100),
                   X_train, y_train)

In [379]:
print('Gradient Boosting Regression best score: ' + str(np.mean(gb_rmse)))

Gradient Boosting Regression best score: 0.3034719322867967


### XGBoost tree:

In [1262]:
def score_XGB_tree(params):
    print("Training with params:")
    print(params)
    params['max_depth'] = int(params['max_depth'])
    dtrain = xgb.DMatrix(X_train_gb, label=y_train_gb)
    dvalid = xgb.DMatrix(X_valid_gb, label=y_valid_gb)
    model = xgb.train(params, dtrain, params['num_round'])
    predictions = model.predict(dvalid)
    score = np.sqrt(mean_squared_error(y_valid_gb, predictions))
    print("\tScore {0}\n\n".format(score))
    loss = score
    return {'loss': loss, 'status': STATUS_OK}


def optimize_XGB_tree(trials):
    space = {
        'booster': 'gbtree',
        'learning_rate': hp.quniform('learning_rate', 0.005, 0.5, 0.005),
        'gamma': hp.quniform('gamma', 0, 10, 1),
        'max_depth': hp.quniform('max_depth', 2, 20, 1),
        'min_child_weight': hp.quniform('min_child_weight', 0.1, 1, 0.05),
        'subsample': hp.quniform('subsample', 0.5, 1, 0.05),
        'colsample_bytree': hp.quniform('colsample_bytree', 0, 1, 0.05),
        'colsample_bylevel': hp.quniform('colsample_bylevel', 0, 1, 0.05),
        'colsample_bynode': hp.quniform('colsample_bynode', 0, 1, 0.05),
        'reg_lambda': hp.quniform('reg_lambda', 0.1, 0.9, 0.05),
        'alpha': hp.quniform('alpha', 0.1, 0.9, 0.05),
        'objective': 'reg:linear',
        'eval_metric': 'rmse',
        'seed': 42,
        'num_round': 500,
        'nthread': -1,
        'silent': 1
    }

    best = fmin(score_XGB_tree, space, algo=tpe.suggest,
                trials=trials, max_evals=1000)
    return best

In [1263]:
trials = Trials()
best_params_XGB_tree = optimize_XGB_tree(trials)
best_params_XGB_tree

Training with params:                                 
{'alpha': 0.75, 'booster': 'gbtree', 'colsample_bylevel': 0.55, 'colsample_bynode': 0.9500000000000001, 'colsample_bytree': 0.9, 'eval_metric': 'rmse', 'gamma': 4.0, 'learning_rate': 0.24, 'max_depth': 15.0, 'min_child_weight': 0.55, 'nthread': -1, 'num_round': 500, 'objective': 'reg:linear', 'reg_lambda': 0.7000000000000001, 'seed': 42, 'silent': 1, 'subsample': 0.75}
	Score 0.20085019861904782                            


Training with params:                                                             
{'alpha': 0.75, 'booster': 'gbtree', 'colsample_bylevel': 0.4, 'colsample_bynode': 0.45, 'colsample_bytree': 0.45, 'eval_metric': 'rmse', 'gamma': 3.0, 'learning_rate': 0.42, 'max_depth': 14.0, 'min_child_weight': 0.2, 'nthread': -1, 'num_round': 500, 'objective': 'reg:linear', 'reg_lambda': 0.75, 'seed': 42, 'silent': 1, 'subsample': 0.8500000000000001}
	Score 0.18561499682580007                                                  

	Score 0.1624111417305374                                                          


Training with params:                                                              
{'alpha': 0.4, 'booster': 'gbtree', 'colsample_bylevel': 0.5, 'colsample_bynode': 0.5, 'colsample_bytree': 0.9500000000000001, 'eval_metric': 'rmse', 'gamma': 5.0, 'learning_rate': 0.09, 'max_depth': 12.0, 'min_child_weight': 0.2, 'nthread': -1, 'num_round': 500, 'objective': 'reg:linear', 'reg_lambda': 0.25, 'seed': 42, 'silent': 1, 'subsample': 0.7000000000000001}
	Score 0.2081783082663986                                                          


Training with params:                                                              
{'alpha': 0.30000000000000004, 'booster': 'gbtree', 'colsample_bylevel': 0.55, 'colsample_bynode': 0.8500000000000001, 'colsample_bytree': 0.8500000000000001, 'eval_metric': 'rmse', 'gamma': 2.0, 'learning_rate': 0.03, 'max_depth': 8.0, 'min_child_weight': 0.35000000000000003, 'nthread': -1

	Score 0.14003108944320208                                                         


Training with params:                                                              
{'alpha': 0.4, 'booster': 'gbtree', 'colsample_bylevel': 0.65, 'colsample_bynode': 0.45, 'colsample_bytree': 1.0, 'eval_metric': 'rmse', 'gamma': 8.0, 'learning_rate': 0.21, 'max_depth': 5.0, 'min_child_weight': 0.45, 'nthread': -1, 'num_round': 500, 'objective': 'reg:linear', 'reg_lambda': 0.65, 'seed': 42, 'silent': 1, 'subsample': 0.7000000000000001}
	Score 0.2345365603513442                                                          


Training with params:                                                              
{'alpha': 0.30000000000000004, 'booster': 'gbtree', 'colsample_bylevel': 0.25, 'colsample_bynode': 0.55, 'colsample_bytree': 0.6000000000000001, 'eval_metric': 'rmse', 'gamma': 5.0, 'learning_rate': 0.03, 'max_depth': 11.0, 'min_child_weight': 0.6000000000000001, 'nthread': -1, 'num_round': 500, 'object

	Score 0.1266998505341282                                                          


Training with params:                                                              
{'alpha': 0.8500000000000001, 'booster': 'gbtree', 'colsample_bylevel': 0.8500000000000001, 'colsample_bynode': 0.8, 'colsample_bytree': 0.25, 'eval_metric': 'rmse', 'gamma': 3.0, 'learning_rate': 0.02, 'max_depth': 6.0, 'min_child_weight': 0.4, 'nthread': -1, 'num_round': 500, 'objective': 'reg:linear', 'reg_lambda': 0.45, 'seed': 42, 'silent': 1, 'subsample': 0.9500000000000001}
	Score 0.1860579942199021                                                         


Training with params:                                                             
{'alpha': 0.9, 'booster': 'gbtree', 'colsample_bylevel': 0.75, 'colsample_bynode': 0.8500000000000001, 'colsample_bytree': 0.45, 'eval_metric': 'rmse', 'gamma': 1.0, 'learning_rate': 0.05, 'max_depth': 5.0, 'min_child_weight': 0.30000000000000004, 'nthread': -1, 'num_round': 50

	Score 0.22586056402642127                                                         


Training with params:                                                              
{'alpha': 0.65, 'booster': 'gbtree', 'colsample_bylevel': 0.9, 'colsample_bynode': 0.9, 'colsample_bytree': 0.7000000000000001, 'eval_metric': 'rmse', 'gamma': 0.0, 'learning_rate': 0.08, 'max_depth': 10.0, 'min_child_weight': 0.4, 'nthread': -1, 'num_round': 500, 'objective': 'reg:linear', 'reg_lambda': 0.45, 'seed': 42, 'silent': 1, 'subsample': 0.6000000000000001}
	Score 0.13440167924258234                                                         


Training with params:                                                              
{'alpha': 0.75, 'booster': 'gbtree', 'colsample_bylevel': 1.0, 'colsample_bynode': 0.75, 'colsample_bytree': 0.1, 'eval_metric': 'rmse', 'gamma': 6.0, 'learning_rate': 0.465, 'max_depth': 2.0, 'min_child_weight': 0.5, 'nthread': -1, 'num_round': 500, 'objective': 'reg:linear', 'reg_lambda'

...

	Score 0.1564290114170966                                                           


Training with params:                                                               
{'alpha': 0.30000000000000004, 'booster': 'gbtree', 'colsample_bylevel': 0.65, 'colsample_bynode': 0.15000000000000002, 'colsample_bytree': 0.75, 'eval_metric': 'rmse', 'gamma': 2.0, 'learning_rate': 0.105, 'max_depth': 2.0, 'min_child_weight': 0.4, 'nthread': -1, 'num_round': 500, 'objective': 'reg:linear', 'reg_lambda': 0.8, 'seed': 42, 'silent': 1, 'subsample': 0.75}
	Score 0.17032029108464497                                                          


Training with params:                                                               
{'alpha': 0.15000000000000002, 'booster': 'gbtree', 'colsample_bylevel': 0.7000000000000001, 'colsample_bynode': 0.1, 'colsample_bytree': 0.9, 'eval_metric': 'rmse', 'gamma': 0.0, 'learning_rate': 0.005, 'max_depth': 3.0, 'min_child_weight': 0.35000000000000003, 'nthread': -1, 'num_

	Score 0.13039885431701476                                                          


Training with params:                                                               
{'alpha': 0.1, 'booster': 'gbtree', 'colsample_bylevel': 0.45, 'colsample_bynode': 0.05, 'colsample_bytree': 0.7000000000000001, 'eval_metric': 'rmse', 'gamma': 1.0, 'learning_rate': 0.04, 'max_depth': 2.0, 'min_child_weight': 0.4, 'nthread': -1, 'num_round': 500, 'objective': 'reg:linear', 'reg_lambda': 0.8, 'seed': 42, 'silent': 1, 'subsample': 0.75}
	Score 0.15374141028196173                                                          


Training with params:                                                               
{'alpha': 0.15000000000000002, 'booster': 'gbtree', 'colsample_bylevel': 0.6000000000000001, 'colsample_bynode': 0.2, 'colsample_bytree': 0.9500000000000001, 'eval_metric': 'rmse', 'gamma': 0.0, 'learning_rate': 0.105, 'max_depth': 4.0, 'min_child_weight': 0.45, 'nthread': -1, 'num_round': 500, 'obje

	Score 0.1271119043463468                                                           


Training with params:                                                               
{'alpha': 0.15000000000000002, 'booster': 'gbtree', 'colsample_bylevel': 0.0, 'colsample_bynode': 0.2, 'colsample_bytree': 0.9500000000000001, 'eval_metric': 'rmse', 'gamma': 1.0, 'learning_rate': 0.37, 'max_depth': 2.0, 'min_child_weight': 0.55, 'nthread': -1, 'num_round': 500, 'objective': 'reg:linear', 'reg_lambda': 0.8, 'seed': 42, 'silent': 1, 'subsample': 0.8}
	Score 0.1919948832899921                                                           


Training with params:                                                               
{'alpha': 0.25, 'booster': 'gbtree', 'colsample_bylevel': 0.6000000000000001, 'colsample_bynode': 0.05, 'colsample_bytree': 0.75, 'eval_metric': 'rmse', 'gamma': 0.0, 'learning_rate': 0.005, 'max_depth': 5.0, 'min_child_weight': 0.30000000000000004, 'nthread': -1, 'num_round': 500, 'obj

	Score 0.169818945311512                                                            


Training with params:                                                               
{'alpha': 0.2, 'booster': 'gbtree', 'colsample_bylevel': 0.7000000000000001, 'colsample_bynode': 0.0, 'colsample_bytree': 0.7000000000000001, 'eval_metric': 'rmse', 'gamma': 0.0, 'learning_rate': 0.155, 'max_depth': 3.0, 'min_child_weight': 0.25, 'nthread': -1, 'num_round': 500, 'objective': 'reg:linear', 'reg_lambda': 0.4, 'seed': 42, 'silent': 1, 'subsample': 0.8}
	Score 0.1272886080746447                                                           


Training with params:                                                               
{'alpha': 0.30000000000000004, 'booster': 'gbtree', 'colsample_bylevel': 0.8, 'colsample_bynode': 0.1, 'colsample_bytree': 0.4, 'eval_metric': 'rmse', 'gamma': 0.0, 'learning_rate': 0.01, 'max_depth': 2.0, 'min_child_weight': 0.4, 'nthread': -1, 'num_round': 500, 'objective': 'reg:linea

{'alpha': 0.1,
 'colsample_bylevel': 0.6000000000000001,
 'colsample_bynode': 0.15000000000000002,
 'colsample_bytree': 0.55,
 'gamma': 0.0,
 'learning_rate': 0.095,
 'max_depth': 2.0,
 'min_child_weight': 0.2,
 'reg_lambda': 0.8500000000000001,
 'subsample': 0.8}

In [1264]:
best_params_XGB_tree['max_depth'] = int(best_params_XGB_tree['max_depth'])

In [1265]:
[el + '=' + str(best_params_XGB_tree[el]) for el in best_params_XGB_tree]

['alpha=0.1',
 'colsample_bylevel=0.6000000000000001',
 'colsample_bynode=0.15000000000000002',
 'colsample_bytree=0.55',
 'gamma=0.0',
 'learning_rate=0.095',
 'max_depth=2',
 'min_child_weight=0.2',
 'reg_lambda=0.8500000000000001',
 'subsample=0.8']

First trial:  
best score: 0.31112249530658054  
['alpha=0.8',
 'colsample_bylevel=0.4',
 'colsample_bynode=0.1',
 'colsample_bytree=0.9500000000000001',
 'gamma=0.0',
 'lambda=0.30000000000000004',
 'learning_rate=0.125',
 'max_depth=2',
 'min_child_weight=0.9500000000000001',
 'subsample=1.0']

Second:  
best score: 0.11873355586847814  
{'alpha': 0.1, 'booster': 'gbtree', 'colsample_bylevel': 0.6000000000000001, 'colsample_bynode': 0.15000000000000002, 'colsample_bytree': 0.55, 'eval_metric': 'rmse', 'gamma': 0.0, 'learning_rate': 0.095, 'max_depth': 2.0, 'min_child_weight': 0.2, 'nthread': -1, 'num_round': 500, 'objective': 'reg:linear', 'reg_lambda': 0.8500000000000001, 'seed': 42, 'silent': 1, 'subsample': 0.8}

In [393]:
xgb_tree_rmse = rmsle_cv(xgb.XGBRegressor(n_estimators=3000,
                                          alpha=0.8,
                                          colsample_bylevel=0.4,
                                          colsample_bynode=0.1,
                                          colsample_bytree=0.9500000000000001,
                                          gamma=0.0,
                                          reg_lambda=0.3000000000000000,
                                          learning_rate=0.125,
                                          max_depth=2,
                                          min_child_weight=0.9500000000000001,
                                          subsample=1.0,
                                          random_state=42),
                         X_train, y_train)

In [394]:
print('XGB tree best score: ' + str(np.mean(xgb_tree_rmse)))

XGB tree best score: 0.31112249530658054


### XGBoost linear:

In [1266]:
def score_XGB_linear(params):
    print("Training with params:")
    print(params)
    dtrain = xgb.DMatrix(X_train_gb, label=y_train_gb)
    dvalid = xgb.DMatrix(X_valid_gb, label=y_valid_gb)
    model = xgb.train(params, dtrain, params['num_round'])
    predictions = model.predict(dvalid)
    score = np.sqrt(mean_squared_error(y_valid_gb, predictions))
    print("\tScore {0}\n\n".format(score))
    loss = score
    return {'loss': loss, 'status': STATUS_OK}


def optimize_XGB_linear(trials):
    space = {
        'booster': 'gblinear',
        'reg_lambda': hp.quniform('reg_lambda', 0.1, 0.9, 0.05),
        'alpha': hp.quniform('alpha', 0.1, 0.9, 0.05),
        'feature_selector': hp.choice('feature_selector', ['cyclic', 'shuffle']),
        'objective': 'reg:linear',
        'eval_metric': 'rmse',
        'seed': 42,
        'num_round': 500,
        'nthread': -1,
        'silent': 1
    }

    best = fmin(score_XGB_linear, space, algo=tpe.suggest,
                trials=trials, max_evals=1000)
    return best

In [1267]:
trials = Trials()
best_params_XGB_linear = optimize_XGB_linear(trials)
best_params_XGB_linear

Training with params:                                 
{'alpha': 0.55, 'booster': 'gblinear', 'eval_metric': 'rmse', 'feature_selector': 'cyclic', 'nthread': -1, 'num_round': 500, 'objective': 'reg:linear', 'reg_lambda': 0.15000000000000002, 'seed': 42, 'silent': 1}
	Score 0.3965651050039765                             


Training with params:                                                            
{'alpha': 0.30000000000000004, 'booster': 'gblinear', 'eval_metric': 'rmse', 'feature_selector': 'shuffle', 'nthread': -1, 'num_round': 500, 'objective': 'reg:linear', 'reg_lambda': 0.4, 'seed': 42, 'silent': 1}
	Score 0.3810559348341266                                                        


Training with params:                                                            
{'alpha': 0.35000000000000003, 'booster': 'gblinear', 'eval_metric': 'rmse', 'feature_selector': 'shuffle', 'nthread': -1, 'num_round': 500, 'objective': 'reg:linear', 'reg_lambda': 0.65, 'seed': 42, 'silent': 1}
	Sc

	Score 0.35470640488872257                                                         


Training with params:                                                              
{'alpha': 0.35000000000000003, 'booster': 'gblinear', 'eval_metric': 'rmse', 'feature_selector': 'shuffle', 'nthread': -1, 'num_round': 500, 'objective': 'reg:linear', 'reg_lambda': 0.2, 'seed': 42, 'silent': 1}
	Score 0.3965651050039765                                                          


Training with params:                                                              
{'alpha': 0.5, 'booster': 'gblinear', 'eval_metric': 'rmse', 'feature_selector': 'shuffle', 'nthread': -1, 'num_round': 500, 'objective': 'reg:linear', 'reg_lambda': 0.55, 'seed': 42, 'silent': 1}
	Score 0.3965651050039765                                                          


Training with params:                                                              
{'alpha': 0.45, 'booster': 'gblinear', 'eval_metric': 'rmse', 'feature_selector':

Training with params:                                                              
{'alpha': 0.25, 'booster': 'gblinear', 'eval_metric': 'rmse', 'feature_selector': 'shuffle', 'nthread': -1, 'num_round': 500, 'objective': 'reg:linear', 'reg_lambda': 0.15000000000000002, 'seed': 42, 'silent': 1}
	Score 0.34140893890334123                                                         


Training with params:                                                              
{'alpha': 0.6000000000000001, 'booster': 'gblinear', 'eval_metric': 'rmse', 'feature_selector': 'shuffle', 'nthread': -1, 'num_round': 500, 'objective': 'reg:linear', 'reg_lambda': 0.25, 'seed': 42, 'silent': 1}
	Score 0.3965651050039765                                                          


Training with params:                                                              
{'alpha': 0.30000000000000004, 'booster': 'gblinear', 'eval_metric': 'rmse', 'feature_selector': 'shuffle', 'nthread': -1, 'num_round': 500, 'objective

	Score 0.20535633996989497                                                          


Training with params:                                                               
{'alpha': 0.1, 'booster': 'gblinear', 'eval_metric': 'rmse', 'feature_selector': 'shuffle', 'nthread': -1, 'num_round': 500, 'objective': 'reg:linear', 'reg_lambda': 0.15000000000000002, 'seed': 42, 'silent': 1}
	Score 0.20717506671275152                                                          


Training with params:                                                               
{'alpha': 0.65, 'booster': 'gblinear', 'eval_metric': 'rmse', 'feature_selector': 'cyclic', 'nthread': -1, 'num_round': 500, 'objective': 'reg:linear', 'reg_lambda': 0.30000000000000004, 'seed': 42, 'silent': 1}
	Score 0.3965651050039765                                                           


Training with params:                                                               
{'alpha': 0.15000000000000002, 'booster': 'gblinear', 'eval_

	Score 0.2521621104982003                                                           


Training with params:                                                               
{'alpha': 0.1, 'booster': 'gblinear', 'eval_metric': 'rmse', 'feature_selector': 'shuffle', 'nthread': -1, 'num_round': 500, 'objective': 'reg:linear', 'reg_lambda': 0.6000000000000001, 'seed': 42, 'silent': 1}
	Score 0.2213084876093038                                                           


Training with params:                                                               
{'alpha': 0.2, 'booster': 'gblinear', 'eval_metric': 'rmse', 'feature_selector': 'cyclic', 'nthread': -1, 'num_round': 500, 'objective': 'reg:linear', 'reg_lambda': 0.2, 'seed': 42, 'silent': 1}
	Score 0.30162454822392526                                                          


Training with params:                                                               
{'alpha': 0.30000000000000004, 'booster': 'gblinear', 'eval_metric': 'rmse', '

...

	Score 0.3965651050039765                                                          


Training with params:                                                              
{'alpha': 0.1, 'booster': 'gblinear', 'eval_metric': 'rmse', 'feature_selector': 'cyclic', 'nthread': -1, 'num_round': 500, 'objective': 'reg:linear', 'reg_lambda': 0.1, 'seed': 42, 'silent': 1}
	Score 0.2053561935947925                                                          


Training with params:                                                              
{'alpha': 0.2, 'booster': 'gblinear', 'eval_metric': 'rmse', 'feature_selector': 'cyclic', 'nthread': -1, 'num_round': 500, 'objective': 'reg:linear', 'reg_lambda': 0.15000000000000002, 'seed': 42, 'silent': 1}
	Score 0.2996151559744248                                                          


Training with params:                                                              
{'alpha': 0.1, 'booster': 'gblinear', 'eval_metric': 'rmse', 'feature_selector': 'cy

	Score 0.20717494259965227                                                         


Training with params:                                                              
{'alpha': 0.15000000000000002, 'booster': 'gblinear', 'eval_metric': 'rmse', 'feature_selector': 'cyclic', 'nthread': -1, 'num_round': 500, 'objective': 'reg:linear', 'reg_lambda': 0.8, 'seed': 42, 'silent': 1}
	Score 0.2733934662026999                                                          


Training with params:                                                              
{'alpha': 0.2, 'booster': 'gblinear', 'eval_metric': 'rmse', 'feature_selector': 'cyclic', 'nthread': -1, 'num_round': 500, 'objective': 'reg:linear', 'reg_lambda': 0.1, 'seed': 42, 'silent': 1}
	Score 0.29690030672198503                                                         


Training with params:                                                              
{'alpha': 0.25, 'booster': 'gblinear', 'eval_metric': 'rmse', 'feature_selector': 'c

	Score 0.20717494259965227                                                         


Training with params:                                                              
{'alpha': 0.30000000000000004, 'booster': 'gblinear', 'eval_metric': 'rmse', 'feature_selector': 'cyclic', 'nthread': -1, 'num_round': 500, 'objective': 'reg:linear', 'reg_lambda': 0.35000000000000003, 'seed': 42, 'silent': 1}
	Score 0.3804840340812258                                                          


Training with params:                                                              
{'alpha': 0.1, 'booster': 'gblinear', 'eval_metric': 'rmse', 'feature_selector': 'cyclic', 'nthread': -1, 'num_round': 500, 'objective': 'reg:linear', 'reg_lambda': 0.1, 'seed': 42, 'silent': 1}
	Score 0.2053561935947925                                                          


Training with params:                                                              
{'alpha': 0.15000000000000002, 'booster': 'gblinear', 'eval_metric':

	Score 0.216761846319728                                                           


Training with params:                                                              
{'alpha': 0.25, 'booster': 'gblinear', 'eval_metric': 'rmse', 'feature_selector': 'cyclic', 'nthread': -1, 'num_round': 500, 'objective': 'reg:linear', 'reg_lambda': 0.1, 'seed': 42, 'silent': 1}
	Score 0.33978359439424377                                                         


Training with params:                                                              
{'alpha': 0.15000000000000002, 'booster': 'gblinear', 'eval_metric': 'rmse', 'feature_selector': 'cyclic', 'nthread': -1, 'num_round': 500, 'objective': 'reg:linear', 'reg_lambda': 0.2, 'seed': 42, 'silent': 1}
	Score 0.25434447436759156                                                         


Training with params:                                                              
{'alpha': 0.30000000000000004, 'booster': 'gblinear', 'eval_metric': 'rmse', 'featu

{'alpha': 0.1, 'feature_selector': 0, 'reg_lambda': 0.1}

In [1268]:
[el + '=' + str(best_params_XGB_linear[el]) for el in best_params_XGB_linear]

['alpha=0.1', 'feature_selector=0', 'reg_lambda=0.1']

First trial:  
score: 0.2979  
n_estimators=3000, alpha=0.1, booster='gblinear', eval_metric='rmse', feature_selector='shuffle', nthread= -1, num_round=500, objective='reg:linear', reg_lambda=0.1, seed=42, silent=1  

Second:  
best score: 0.2053561935947925  
{'alpha': 0.1, 'booster': 'gblinear', 'eval_metric': 'rmse', 'feature_selector': 'cyclic', 'nthread': -1, 'num_round': 500, 'objective': 'reg:linear', 'reg_lambda': 0.1, 'seed': 42, 'silent': 1}
	Score 0.2053561935947925

In [1152]:
xgb_linear_rmse = rmsle_cv(xgb.XGBRegressor(n_estimators=3000,
                                            alpha=0.1,
                                            booster='gblinear',
                                            eval_metric='rmse',
                                            feature_selector='shuffle',
                                            nthread=-1,
                                            num_round=500,
                                            objective='reg:linear',
                                            reg_lambda=0.1,
                                            seed=42,
                                            silent=1),
                           X_train, y_train)

In [1153]:
print('XGB linear best score: ' + str(np.mean(xgb_linear_rmse)))

XGB linear best score: 0.12938484436057354


### LightGBM:

In [1269]:
def score_LGB(params):
    print("Training with params:")
    print(params)
    params['num_leaves'] = int(params['num_leaves'])
    params['max_depth'] = int(params['max_depth'])
    params['min_data_in_leaf'] = int(params['min_data_in_leaf'])
    params['bagging_freq'] = int(params['bagging_freq'])
    params['max_bin'] = int(params['max_bin'])
    dtrain = lgb.Dataset(X_train_gb, label=y_train_gb)
    dvalid = lgb.Dataset(X_valid_gb, label=y_valid_gb)
    model = lgb.train(params, dtrain, params['num_round'])
    predictions = model.predict(X_valid_gb)
    score = np.sqrt(mean_squared_error(y_valid_gb, predictions))
    print("\tScore {0}\n\n".format(score))
    loss = score
    return {'loss': loss, 'status': STATUS_OK}


def optimize_LGB(trials):
    space = {
        'objective': 'huber',
        'boosting': 'gbdt',
        'num_round': 500,
        'learning_rate': hp.quniform('learning_rate', 0.005, 0.5, 0.005),
        'num_leaves': hp.quniform('num_leaves', 2, 40, 1),
        'nthread': 4,
        'seed': 42,
        'max_depth': hp.quniform('max_depth', 2, 20, 1),
        'min_data_in_leaf': hp.quniform('min_data_in_leaf', 2, 30, 1),
        'min_sum_hessian_in_leaf': hp.quniform('min_sum_hessian_in_leaf', 0.001, 30, 1),
        'subsample': hp.quniform('subsample', 0.3, 1, 0.05),
        'bagging_freq': hp.quniform('bagging_freq', 0, 10, 1),
        'bagging_seed': 42,
        'feature_fraction': hp.quniform('colsample_bytree', 0.1, 1, 0.05),
        'feature_fraction_seed': 42,
        'lambda_l1': hp.quniform('lambda_l1', 0.1, 0.9, 0.05),
        'lambda_l2': hp.quniform('lambda_l2', 0.1, 0.9, 0.05),
        'max_bin': hp.quniform('max_bin', 15, 255, 20),
        'metric': 'rmse'
    }

    best = fmin(score_LGB, space, algo=tpe.suggest,
                trials=trials, max_evals=1000)
    return best

In [1270]:
trials = Trials()
best_params_LGB = optimize_LGB(trials)
best_params_LGB

Training with params:                                 
{'bagging_freq': 8.0, 'bagging_seed': 42, 'boosting': 'gbdt', 'feature_fraction': 0.1, 'feature_fraction_seed': 42, 'lambda_l1': 0.9, 'lambda_l2': 0.35000000000000003, 'learning_rate': 0.36, 'max_bin': 80.0, 'max_depth': 18.0, 'metric': 'rmse', 'min_data_in_leaf': 24.0, 'min_sum_hessian_in_leaf': 26.0, 'nthread': 4, 'num_leaves': 13.0, 'num_round': 500, 'objective': 'huber', 'seed': 42, 'subsample': 0.55}
	Score 0.14269334108104917                            


Training with params:                                                             
{'bagging_freq': 10.0, 'bagging_seed': 42, 'boosting': 'gbdt', 'feature_fraction': 0.8, 'feature_fraction_seed': 42, 'lambda_l1': 0.8, 'lambda_l2': 0.8500000000000001, 'learning_rate': 0.375, 'max_bin': 80.0, 'max_depth': 7.0, 'metric': 'rmse', 'min_data_in_leaf': 25.0, 'min_sum_hessian_in_leaf': 23.0, 'nthread': 4, 'num_leaves': 12.0, 'num_round': 500, 'objective': 'huber', 'seed': 42, 'subsa

Training with params:                                                             
{'bagging_freq': 6.0, 'bagging_seed': 42, 'boosting': 'gbdt', 'feature_fraction': 0.45, 'feature_fraction_seed': 42, 'lambda_l1': 0.4, 'lambda_l2': 0.35000000000000003, 'learning_rate': 0.47500000000000003, 'max_bin': 140.0, 'max_depth': 10.0, 'metric': 'rmse', 'min_data_in_leaf': 28.0, 'min_sum_hessian_in_leaf': 10.0, 'nthread': 4, 'num_leaves': 15.0, 'num_round': 500, 'objective': 'huber', 'seed': 42, 'subsample': 0.9}
	Score 0.15806317523672345                                                        


Training with params:                                                             
{'bagging_freq': 1.0, 'bagging_seed': 42, 'boosting': 'gbdt', 'feature_fraction': 0.30000000000000004, 'feature_fraction_seed': 42, 'lambda_l1': 0.8, 'lambda_l2': 0.4, 'learning_rate': 0.19, 'max_bin': 180.0, 'max_depth': 6.0, 'metric': 'rmse', 'min_data_in_leaf': 24.0, 'min_sum_hessian_in_leaf': 26.0, 'nthread': 4, 'num_l

	Score 0.15797547679051788                                                       


Training with params:                                                              
{'bagging_freq': 2.0, 'bagging_seed': 42, 'boosting': 'gbdt', 'feature_fraction': 0.9, 'feature_fraction_seed': 42, 'lambda_l1': 0.65, 'lambda_l2': 0.75, 'learning_rate': 0.055, 'max_bin': 220.0, 'max_depth': 18.0, 'metric': 'rmse', 'min_data_in_leaf': 24.0, 'min_sum_hessian_in_leaf': 14.0, 'nthread': 4, 'num_leaves': 5.0, 'num_round': 500, 'objective': 'huber', 'seed': 42, 'subsample': 0.30000000000000004}
	Score 0.13322749981026413                                                         


Training with params:                                                              
{'bagging_freq': 1.0, 'bagging_seed': 42, 'boosting': 'gbdt', 'feature_fraction': 0.9500000000000001, 'feature_fraction_seed': 42, 'lambda_l1': 0.4, 'lambda_l2': 0.55, 'learning_rate': 0.48, 'max_bin': 260.0, 'max_depth': 15.0, 'metric': 'rmse', 'min_

	Score 0.1493883081579818                                                          


Training with params:                                                              
{'bagging_freq': 2.0, 'bagging_seed': 42, 'boosting': 'gbdt', 'feature_fraction': 0.45, 'feature_fraction_seed': 42, 'lambda_l1': 0.7000000000000001, 'lambda_l2': 0.6000000000000001, 'learning_rate': 0.39, 'max_bin': 260.0, 'max_depth': 19.0, 'metric': 'rmse', 'min_data_in_leaf': 15.0, 'min_sum_hessian_in_leaf': 19.0, 'nthread': 4, 'num_leaves': 6.0, 'num_round': 500, 'objective': 'huber', 'seed': 42, 'subsample': 0.7000000000000001}
	Score 0.1441466013139166                                                          


Training with params:                                                              
{'bagging_freq': 0.0, 'bagging_seed': 42, 'boosting': 'gbdt', 'feature_fraction': 0.9, 'feature_fraction_seed': 42, 'lambda_l1': 0.5, 'lambda_l2': 0.8500000000000001, 'learning_rate': 0.495, 'max_bin': 240.0, 'max_depth': 

	Score 0.12744672571389354                                                         


Training with params:                                                              
{'bagging_freq': 5.0, 'bagging_seed': 42, 'boosting': 'gbdt', 'feature_fraction': 0.2, 'feature_fraction_seed': 42, 'lambda_l1': 0.2, 'lambda_l2': 0.15000000000000002, 'learning_rate': 0.075, 'max_bin': 100.0, 'max_depth': 8.0, 'metric': 'rmse', 'min_data_in_leaf': 12.0, 'min_sum_hessian_in_leaf': 2.0, 'nthread': 4, 'num_leaves': 8.0, 'num_round': 500, 'objective': 'huber', 'seed': 42, 'subsample': 0.75}
	Score 0.13242526307900973                                                         


Training with params:                                                              
{'bagging_freq': 6.0, 'bagging_seed': 42, 'boosting': 'gbdt', 'feature_fraction': 0.4, 'feature_fraction_seed': 42, 'lambda_l1': 0.15000000000000002, 'lambda_l2': 0.25, 'learning_rate': 0.135, 'max_bin': 100.0, 'max_depth': 10.0, 'metric': 'rmse', 'min

...

	Score 0.13441978793123985                                                          


Training with params:                                                               
{'bagging_freq': 4.0, 'bagging_seed': 42, 'boosting': 'gbdt', 'feature_fraction': 0.4, 'feature_fraction_seed': 42, 'lambda_l1': 0.1, 'lambda_l2': 0.2, 'learning_rate': 0.27, 'max_bin': 260.0, 'max_depth': 6.0, 'metric': 'rmse', 'min_data_in_leaf': 4.0, 'min_sum_hessian_in_leaf': 6.0, 'nthread': 4, 'num_leaves': 2.0, 'num_round': 500, 'objective': 'huber', 'seed': 42, 'subsample': 0.75}
	Score 0.124227588218217                                                            


Training with params:                                                               
{'bagging_freq': 5.0, 'bagging_seed': 42, 'boosting': 'gbdt', 'feature_fraction': 0.6000000000000001, 'feature_fraction_seed': 42, 'lambda_l1': 0.9, 'lambda_l2': 0.1, 'learning_rate': 0.245, 'max_bin': 260.0, 'max_depth': 2.0, 'metric': 'rmse', 'min_data_in_leaf': 5

	Score 0.14086419160529942                                                          


Training with params:                                                               
{'bagging_freq': 3.0, 'bagging_seed': 42, 'boosting': 'gbdt', 'feature_fraction': 0.7000000000000001, 'feature_fraction_seed': 42, 'lambda_l1': 0.1, 'lambda_l2': 0.2, 'learning_rate': 0.295, 'max_bin': 260.0, 'max_depth': 6.0, 'metric': 'rmse', 'min_data_in_leaf': 2.0, 'min_sum_hessian_in_leaf': 1.0, 'nthread': 4, 'num_leaves': 6.0, 'num_round': 500, 'objective': 'huber', 'seed': 42, 'subsample': 0.6000000000000001}
	Score 0.13626877030965762                                                          


Training with params:                                                               
{'bagging_freq': 7.0, 'bagging_seed': 42, 'boosting': 'gbdt', 'feature_fraction': 0.5, 'feature_fraction_seed': 42, 'lambda_l1': 0.15000000000000002, 'lambda_l2': 0.15000000000000002, 'learning_rate': 0.2, 'max_bin': 260.0, 'max_depth':

	Score 0.14443911431536297                                                          


Training with params:                                                               
{'bagging_freq': 3.0, 'bagging_seed': 42, 'boosting': 'gbdt', 'feature_fraction': 0.35000000000000003, 'feature_fraction_seed': 42, 'lambda_l1': 0.15000000000000002, 'lambda_l2': 0.15000000000000002, 'learning_rate': 0.29, 'max_bin': 260.0, 'max_depth': 4.0, 'metric': 'rmse', 'min_data_in_leaf': 7.0, 'min_sum_hessian_in_leaf': 3.0, 'nthread': 4, 'num_leaves': 5.0, 'num_round': 500, 'objective': 'huber', 'seed': 42, 'subsample': 0.65}
	Score 0.14150615310108233                                                          


Training with params:                                                               
{'bagging_freq': 3.0, 'bagging_seed': 42, 'boosting': 'gbdt', 'feature_fraction': 0.4, 'feature_fraction_seed': 42, 'lambda_l1': 0.15000000000000002, 'lambda_l2': 0.25, 'learning_rate': 0.23500000000000001, 'max_bin': 

	Score 0.1392728565322929                                                           


Training with params:                                                               
{'bagging_freq': 6.0, 'bagging_seed': 42, 'boosting': 'gbdt', 'feature_fraction': 0.45, 'feature_fraction_seed': 42, 'lambda_l1': 0.2, 'lambda_l2': 0.15000000000000002, 'learning_rate': 0.265, 'max_bin': 260.0, 'max_depth': 4.0, 'metric': 'rmse', 'min_data_in_leaf': 2.0, 'min_sum_hessian_in_leaf': 4.0, 'nthread': 4, 'num_leaves': 3.0, 'num_round': 500, 'objective': 'huber', 'seed': 42, 'subsample': 0.6000000000000001}
	Score 0.1322919351789858                                                           


Training with params:                                                               
{'bagging_freq': 2.0, 'bagging_seed': 42, 'boosting': 'gbdt', 'feature_fraction': 0.30000000000000004, 'feature_fraction_seed': 42, 'lambda_l1': 0.15000000000000002, 'lambda_l2': 0.25, 'learning_rate': 0.32, 'max_bin': 260.0, 'max_dep

{'bagging_freq': 6.0,
 'colsample_bytree': 0.65,
 'lambda_l1': 0.1,
 'lambda_l2': 0.1,
 'learning_rate': 0.28,
 'max_bin': 260.0,
 'max_depth': 3.0,
 'min_data_in_leaf': 2.0,
 'min_sum_hessian_in_leaf': 1.0,
 'num_leaves': 2.0,
 'subsample': 0.75}

First trial:  
best score: 0.30602079343781113  
'bagging_freq': 6.0, 'bagging_seed': 42, 'boosting': 'gbdt', 'feature_fraction': 0.7000000000000001, 'feature_fraction_seed': 42, 'lambda_l1': 0.55, 'lambda_l2': 0.9, 'learning_rate': 0.045, 'max_bin': 260.0, 'max_depth': 2.0, 'metric': 'rmse', 'min_data_in_leaf': 16.0, 'min_sum_hessian_in_leaf': 10.0, 'nthread': 4, 'num_leaves': 6.0, 'num_round': 500, 'objective': 'huber', 'seed': 42, 'subsample': 0.8500000000000001

Second:  
{'bagging_freq': 6.0, 'bagging_seed': 42, 'boosting': 'gbdt', 'feature_fraction': 0.65, 'feature_fraction_seed': 42, 'lambda_l1': 0.1, 'lambda_l2': 0.1, 'learning_rate': 0.28, 'max_bin': 260.0, 'max_depth': 3.0, 'metric': 'rmse', 'min_data_in_leaf': 2.0, 'min_sum_hessian_in_leaf': 1.0, 'nthread': 4, 'num_leaves': 2.0, 'num_round': 500, 'objective': 'huber', 'seed': 42, 'subsample': 0.75}
	Score 0.11957940856119287 

In [438]:
lgb_rmse = rmsle_cv(lgb.LGBMRegressor(n_estimators=3000,
                                      bagging_freq=6,
                                      bagging_seed=42,
                                      boosting='gbdt',
                                      feature_fraction=0.7000000000000001,
                                      feature_fraction_seed=42,
                                      lambda_l1=0.55,
                                      lambda_l2=0.9,
                                      learning_rate=0.045,
                                      max_bin=260,
                                      max_depth=2,
                                      metric='rmse',
                                      min_data_in_leaf=16,
                                      min_sum_hessian_in_leaf=10,
                                      nthread=4,
                                      num_leaves=6,
                                      num_round=500,
                                      objective='huber',
                                      seed=42,
                                      subsample=0.8500000000000001),
                    X_train, y_train)

In [439]:
print('LightGBM best score: ' + str(np.mean(lgb_rmse)))

LightGBM best score: 0.30602079343781113


### **We will use parametrs below to tune features and compare results:**

Save it as markdown to avoid accidentally erasing

**Lasso:**  
score: 0.2834  
alpha=0.00151

**Ridge:**  
score: 0.2916  
alpha=20.86

**Elastic:**   
score: 0.2848  
l1_ratio=0.52, alpha=0.003  

**Kernel Ridge:**  
score: 0.2852  
alpha=2.22, coef0=2, degree=4, kernel=polynomial  

**Gradient Boosting Regression:**  
score: 0.3035  
n_estimators=3000, loss='huber', alpha=0.9, learning_rate=0.05, max_depth=5, max_features=0.75, min_samples_leaf=5, min_samples_split=17, subsample=0.65, random_state=42, n_iter_no_change=100

**XGB tree:**  
score: 0.3111  
n_estimators=3000, alpha=0.8, colsample_bylevel=0.4, colsample_bynode=0.1, colsample_bytree=0.95, gamma=0.0, reg_lambda=0.3, learning_rate=0.125, max_depth=2, min_child_weight=0.95, subsample=1.0, random_state=42  

**XGB linear:**  
score: 0.2979  
n_estimators=3000, alpha=0.1, booster='gblinear', eval_metric='rmse', feature_selector='shuffle', nthread= -1, num_round=500, objective='reg:linear', reg_lambda=0.1, seed=42, silent=1  

**LightGBM:**  
score: 0.3060  
n_estimators=3000, bagging_freq=6, bagging_seed=42, boosting='gbdt', feature_fraction=0.7, feature_fraction_seed=42, lambda_l1=0.55, lambda_l2=0.9, learning_rate=0.045, max_bin=260, max_depth=2, metric='rmse', min_data_in_leaf=16, min_sum_hessian_in_leaf=10, nthread=4, num_leaves=6, num_round=500, objective='huber', seed=42, subsample=0.85

## Second trial params

**Lasso:**  
score: 0.1089
alpha=0.0006

**Ridge:**  
score: 0.1119
alpha=18.987

**Elastic:**   
score: 0.1092
l1_ratio=0.6414, alpha=0.001  

**Kernel Ridge:**  
score: 0.1109  
alpha=1.4142, coef0=3, degree=3, kernel='polynomial'

**Gradient Boosting Regression:**  
Score 0.131  
alpha=0.9, learning_rate=0.045, loss=huber, max_depth=18, max_features=0.4, min_samples_leaf=8, min_samples_split=4, n_estimators=100, subsample=0.6} 

**XGB tree:**  
Score 0.11873355586847814  
alpha=0.1, booster=gbtree, colsample_bylevel=0.6, colsample_bynode=0.15, colsample_bytree=0.55, eval_metric='rmse', gamma=0.0, learning_rate=0.095, max_depth=2, min_child_weight=0.2, nthread=-1, num_round=500, objective='reg:linear', reg_lambda=0.85, seed=42, silent=1, subsample=0.8}
	

**XGB linear:**  
score:0.2053561935947925  
alpha=0.1, booster='gblinear', eval_metric='rmse', feature_selector='cyclic', nthread=-1, num_round=500, objective='reg:linear', reg_lambda=0.1, seed=42, silent=1 

**LightGBM:**  
score: 0.11957940856119287  
bagging_freq=6, bagging_seed=42, boosting='gbdt', feature_fraction=0.65, feature_fraction_seed=42, lambda_l1=0.1, lambda_l2=0.1, learning_rate=0.28, max_bin=260, max_depth=3, metric='rmse', min_data_in_leaf=2, min_sum_hessian_in_leaf=1, nthread=4, num_leaves=2, num_round=500, objective='huber', seed=42, subsample=0.75  

Let's try to average predictions of this models with untuned parametrs and see score on leaderboard. 

In [1650]:
lasso_test = Lasso(alpha=0.0006).fit(X_train, y_train)
# lasso_test = np.expm1(lasso_test.predict(X_test).reshape(-1, 1))
lasso_test = pt_target.inverse_transform(
    lasso_test.predict(X_test).reshape(-1, 1))

In [1651]:
ridge_test = Ridge(alpha=18.987).fit(X_train, y_train)
# ridge_test = np.expm1(ridge_test.predict(X_test).reshape(-1, 1))
ridge_test = pt_target.inverse_transform(
    ridge_test.predict(X_test).reshape(-1, 1))

In [1652]:
elastic_test = ElasticNet(l1_ratio=0.6414,
                          alpha=0.001).fit(X_train, y_train)
# elastic_test = np.expm1(elastic_test.predict(X_test).reshape(-1, 1))
elastic_test = pt_target.inverse_transform(
    elastic_test.predict(X_test).reshape(-1, 1))

In [1653]:
kernel_test = KernelRidge(alpha=1.4142,
                          coef0=3,
                          degree=3,
                          kernel='polynomial').fit(X_train, y_train)
#kernel_test = np.expm1(kernel_test.predict(X_test).reshape(-1, 1))
kernel_test = pt_target.inverse_transform(
    kernel_test.predict(X_test).reshape(-1, 1))

In [1654]:
gb_test = GradientBoostingRegressor(n_estimators=3000,
                                    alpha=0.9,
                                    learning_rate=0.045,
                                    loss='huber',
                                    max_depth=18,
                                    max_features=0.4,
                                    min_samples_leaf=8,
                                    min_samples_split=4,
                                    subsample=0.6,
                                    n_iter_no_change=100).fit(X_train, y_train)

# gb_test = np.expm1(gb_test.predict(X_test).reshape(-1, 1))
gb_test = pt_target.inverse_transform(gb_test.predict(X_test).reshape(-1, 1))

In [1655]:
xgb_test = xgb.XGBRegressor(n_estimators=3000,
                            alpha=0.1,
                            booster='gbtree',
                            colsample_bylevel=0.6,
                            colsample_bynode=0.15,
                            colsample_bytree=0.55,
                            eval_metric='rmse',
                            gamma=0.0,
                            learning_rate=0.095,
                            max_depth=2,
                            min_child_weight=0.2,
                            nthread=-1,
                            num_round=500,
                            objective='reg:linear',
                            reg_lambda=0.85,
                            seed=42,
                            silent=1,
                            subsample=0.8).fit(X_train, y_train)

# xgb_test = np.expm1(xgb_test.predict(X_test).reshape(-1, 1))
xgb_test = pt_target.inverse_transform(xgb_test.predict(X_test).reshape(-1, 1))

In [1656]:
xgb_linear_test = xgb.XGBRegressor(n_estimators=3000,
                                   alpha=0.1,
                                   booster='gblinear',
                                   eval_metric='rmse',
                                   feature_selector='cyclic',
                                   nthread=-1,
                                   num_round=500,
                                   objective='reg:linear',
                                   reg_lambda=0.1,
                                   seed=42,
                                   silent=1).fit(X_train, y_train)

# xgb_linear_test = np.expm1(xgb_linear_test.predict(X_test).reshape(-1, 1))
xgb_linear_test = pt_target.inverse_transform(
    xgb_linear_test.predict(X_test).reshape(-1, 1))

In [1657]:
lgb_test = lgb.LGBMRegressor(n_estimators=3000,
                             bagging_freq=6,
                             bagging_seed=42,
                             boosting='gbdt',
                             feature_fraction=0.65,
                             feature_fraction_seed=42,
                             lambda_l1=0.1,
                             lambda_l2=0.1,
                             learning_rate=0.28,
                             max_bin=260,
                             max_depth=3,
                             metric='rmse',
                             min_data_in_leaf=2,
                             min_sum_hessian_in_leaf=1,
                             nthread=4,
                             num_leaves=2,
                             num_round=500,
                             objective='huber',
                             seed=42,
                             subsample=0.75).fit(X_train, y_train)

# lgb_test = np.expm1(lgb_test.predict(X_test).reshape(-1, 1))
lgb_test = pt_target.inverse_transform(lgb_test.predict(X_test).reshape(-1, 1))

In [1658]:
predicted_test = 0.125 * lasso_test + \
                 0.125 * ridge_test + \
                 0.125 * elastic_test + \
                 0.125 * kernel_test + \
                 0.125 * gb_test + \
                 0.125 * xgb_test + \
                 0.125 * xgb_linear_test + \
                 0.125 * lgb_test


In [1659]:
write_to_submission_file(predicted_test, 'house_pricing_submission.csv')

The score is 0.11552, previous test score was 0.11629

In [1661]:
predicted_test

array([[123710.72506766],
       [158331.20599329],
       [187052.13615824],
       ...,
       [169956.50884226],
       [114772.93992711],
       [220253.87474725]])

# Modelling

We use modified cross_val_score with suffled data before splitting.

## Base models

### Lasso Regression:

In [1512]:
lasso = make_pipeline(RobustScaler(), Lasso(alpha=0.0006,
                                            random_state=42))

### Ridge Regression:

In [1494]:
ridge = make_pipeline(RobustScaler(), Ridge(alpha=18.987,
                                            random_state=42))

### Elastic Net Regression:

In [1495]:
ENet = make_pipeline(RobustScaler(), ElasticNet(l1_ratio=0.6414,
                                                alpha=0.001,
                                                random_state=42))

### Kernel Ridge Regression:

In [1496]:
KRR = KernelRidge(alpha=1.4142,
                  coef0=3,
                  degree=3,
                  kernel='polynomial')

### Gradient Boosting Regression:

With huber loss that makes it robust to outliers

In [1497]:
GBoost = GradientBoostingRegressor(n_estimators=300,
                                   alpha=0.9,
                                   learning_rate=0.045,
                                   loss='huber',
                                   max_depth=18,
                                   max_features=0.4,
                                   min_samples_leaf=8,
                                   min_samples_split=4,
                                   subsample=0.6,
                                   n_iter_no_change=100)

### XGBoost tree:

In [1498]:
model_xgb = xgb.XGBRegressor(n_estimators=300,
                             alpha=0.1,
                             booster='gbtree',
                             colsample_bylevel=0.6,
                             colsample_bynode=0.15,
                             colsample_bytree=0.55,
                             eval_metric='rmse',
                             gamma=0.0,
                             learning_rate=0.095,
                             max_depth=2,
                             min_child_weight=0.2,
                             nthread=-1,
                             num_round=500,
                             objective='reg:linear',
                             reg_lambda=0.85,
                             seed=42,
                             silent=1,
                             subsample=0.8)

### XGBoost linear:

In [1499]:
model_xgb_linear = xgb.XGBRegressor(n_estimators=300,
                                    alpha=0.1,
                                    booster='gblinear',
                                    eval_metric='rmse',
                                    feature_selector='cyclic',
                                    nthread=-1,
                                    num_round=500,
                                    objective='reg:linear',
                                    reg_lambda=0.1,
                                    seed=42,
                                    silent=1)

### LightGBM:

In [1500]:
model_lgb = lgb.LGBMRegressor(n_estimators=300,
                              bagging_freq=6,
                              bagging_seed=42,
                              boosting='gbdt',
                              feature_fraction=0.65,
                              feature_fraction_seed=42,
                              lambda_l1=0.1,
                              lambda_l2=0.1,
                              learning_rate=0.28,
                              max_bin=260,
                              max_depth=3,
                              metric='rmse',
                              min_data_in_leaf=2,
                              min_sum_hessian_in_leaf=1,
                              nthread=4,
                              num_leaves=2,
                              num_round=500,
                              objective='huber',
                              seed=42,
                              subsample=0.75)

**Create dict with scores**

In [1501]:
# scores = {'Lasso': np.array(0),
#           'Ridge': np.array(0),
#           'ElasticNet': np.array(0),
#           'Kernel Ridge': np.array(0),
#           'Gradient Boosting': np.array(0),
#           'Xgboost': np.array(0),
#           'Xgboost linear': np.array(0),
#           'LGBM': np.array(0)}

### Base models scores

In [1615]:
print("\nLasso previous score: {:.4f} ({:.4f})".format(
    scores['Lasso'].mean(), scores['Lasso'].std()))
scores['Lasso'] = rmsle_cv(lasso, X_train, y_train)
print("Lasso actual score:   {:.4f} ({:.4f})\n".format(
    scores['Lasso'].mean(), scores['Lasso'].std()))


Lasso previous score: 0.2826 (0.0158)
Lasso actual score:   0.2826 (0.0158)



In [1616]:
print("\nRidge previous score: {:.4f} ({:.4f})".format(
    scores['Ridge'].mean(), scores['Ridge'].std()))
scores['Ridge'] = rmsle_cv(ridge, X_train, y_train)
print("Ridge actual score:   {:.4f} ({:.4f})\n".format(
    scores['Ridge'].mean(), scores['Ridge'].std()))


Ridge previous score: 0.2854 (0.0140)
Ridge actual score:   0.2854 (0.0140)



In [1617]:
print("\nElasticNet previous score: {:.4f} ({:.4f})".format(
    scores['ElasticNet'].mean(), scores['ElasticNet'].std()))
scores['ElasticNet'] = rmsle_cv(ENet, X_train, y_train)
print("ElasticNet actual score:   {:.4f} ({:.4f})\n".format(
    scores['ElasticNet'].mean(), scores['ElasticNet'].std()))


ElasticNet previous score: 0.2815 (0.0158)
ElasticNet actual score:   0.2815 (0.0158)



In [1618]:
print("\nKernel Ridge previous score: {:.4f} ({:.4f})".format(
    scores['Kernel Ridge'].mean(), scores['Kernel Ridge'].std()))
scores['Kernel Ridge'] = rmsle_cv(KRR, X_train, y_train)
print("Kernel Ridge actual score:   {:.4f} ({:.4f})\n".format(
    scores['Kernel Ridge'].mean(), scores['Kernel Ridge'].std()))


Kernel Ridge previous score: 0.2804 (0.0150)
Kernel Ridge actual score:   0.2804 (0.0150)



In [1619]:
%%time
print("\nGradient Boosting previous score: {:.4f} ({:.4f})".format(
    scores['Gradient Boosting'].mean(), scores['Gradient Boosting'].std()))
scores['Gradient Boosting'] = rmsle_cv(GBoost, X_train, y_train)
print("Gradient Boosting actual score:   {:.4f} ({:.4f})\n".format(
    scores['Gradient Boosting'].mean(), scores['Gradient Boosting'].std()))


Gradient Boosting previous score: 0.3079 (0.0153)
Gradient Boosting actual score:   0.3054 (0.0197)

CPU times: user 38.2 s, sys: 50 ms, total: 38.2 s
Wall time: 38.2 s


In [1620]:
%%time
print("\nXgboost previous score: {:.4f} ({:.4f})".format(
    scores['Xgboost'].mean(), scores['Xgboost'].std()))
scores['Xgboost'] = rmsle_cv(model_xgb, X_train, y_train)
print("Xgboost actual score:   {:.4f} ({:.4f})\n".format(
    scores['Xgboost'].mean(), scores['Xgboost'].std()))


Xgboost previous score: 0.2991 (0.0199)
Xgboost actual score:   0.2991 (0.0199)

CPU times: user 14.4 s, sys: 160 ms, total: 14.6 s
Wall time: 3.89 s


In [1621]:
%%time
print("\nXgboost linear previous score: {:.4f} ({:.4f})".format(
    scores['Xgboost linear'].mean(), scores['Xgboost linear'].std()))
scores['Xgboost linear'] = rmsle_cv(model_xgb_linear, X_train, y_train)
print("Xgboost linear actual score:   {:.4f} ({:.4f})\n".format(
    scores['Xgboost linear'].mean(), scores['Xgboost linear'].std()))


Xgboost linear previous score: 0.2937 (0.0133)
Xgboost linear actual score:   0.2937 (0.0133)

CPU times: user 23.4 s, sys: 320 ms, total: 23.7 s
Wall time: 6.21 s


In [1622]:
%%time
print("\nLGBM previous score: {:.4f} ({:.4f})".format(
    scores['LGBM'].mean(), scores['LGBM'].std()))
scores['LGBM'] = rmsle_cv(model_lgb, X_train, y_train)
print("LGBM actual score:   {:.4f} ({:.4f})\n".format(
    scores['LGBM'].mean(), scores['LGBM'].std()))


LGBM previous score: 0.3034 (0.0126)
LGBM actual score:   0.3034 (0.0126)

CPU times: user 3.42 s, sys: 50 ms, total: 3.47 s
Wall time: 1.03 s
